In [12]:
import boto3
from botocore.client import Config
from dotenv import load_dotenv
import os

# Get credentials from environment variables
load_dotenv("config.env")
minio_access_key = os.environ.get("access_key")
minio_secret_key = os.environ.get("secret_key")
ACCESS_KEY_ID = os.environ.get("ACCESS_KEY_ID")
SECRET_ACCESS_KEY = os.environ.get("SECRET_ACCESS_KEY")

# Initialize S3 resource for CDSE
session = boto3.session.Session()
s3 = session.resource(
    's3',
    endpoint_url='https://eodata.dataspace.copernicus.eu',
    aws_access_key_id=ACCESS_KEY_ID,
    aws_secret_access_key=SECRET_ACCESS_KEY,
    region_name='default'
)

# Initialize S3 resource for MinIO
minio_s3 = boto3.resource(
    's3',
    endpoint_url='https://pangeo-eosc-minioapi.vm.fedcloud.eu',  # MinIO API endpoint:contentReference[oaicite:4]{index=4}
    aws_access_key_id=minio_access_key,
    aws_secret_access_key=minio_secret_key,
    region_name='default',  # or any region; required by boto3 but not used by MinIO
    config=Config(signature_version='s3v4')  # Use V4 signatures for MinIO:contentReference[oaicite:5]{index=5}
)
source_bucket = s3.Bucket("eodata")               # source bucket (Copernicus)
dest_bucket   = minio_s3.Bucket("okihle-s2l1b")   # destination bucket (MinIO)

In [13]:
prefix = "Sentinel-2/MSI/MSI_L1B_GR/2025/07/24/"
for obj in source_bucket.objects.filter(Prefix=prefix):
    if obj.key.endswith("/"):
        continue  # skip folder placeholders
    print(f"Copying {obj.key} ...")
    # Stream from source and upload to destination
    source_response = source_bucket.Object(obj.key).get()
    dest_bucket.upload_fileobj(source_response['Body'], obj.key)

Copying Sentinel-2/MSI/MSI_L1B_GR/2025/07/24/S2A_OPER_MSI_L1B_GR_2APS_20250724T011624_S20250724T000002_D02_N05.11.tar ...


ClientError: An error occurred (InvalidAccessKeyId) when calling the CreateMultipartUpload operation: The Access Key Id you provided does not exist in our records.